Plan de la tâche - Stakeholders' map

1. Création des types de données
2. Création des tables csv
3. Importation dans neo4j sous forme de noeuds-relations
4. Design des requêtes
5. Possibilité d'interroger la base via un notebook Jupyter


# Création de tables CSV

## Design des noms

In [1]:
from random import random
import numpy as np
import pandas as pd

In [2]:
#noms de famille les plus courants en France entre 1891 et 1900
nom = ["Martin", "Bernard", "Thomas", "Petit", "Robert", "Richard", "Durand", "Dubois", "Moreau", "Laurent", "Simon", "Michel", "Lefèvre", "Leroy", "Roux", "David", "Bertrand", "Morel", "Fournier", "Girard", "Bonnet", "Dupont", "Lambert", "Fontaine", "Rousseau", "Vincent", "Muller", "Lefevre", "Faure", "Andre", "Mercier", "Blanc", "Guerin", "Boyer", "Garnier", "Chevalier", "Francois", "Legrand", "Gauthier", "Garcia", "Perrin", "Robin", "Clement", "Morin", "Nicolas", "Henry", "Roussel", "Mathieu", "Gautier", "Masson", "Marchand", "Duval", "Denis", "Dumont", "Marie", "Lemaire", "Noel", "Meyer", "Dufour", "Meunier", "Brun", "Blanchard", "Giraud", "Joly", "Riviere", "Lucas", "Brunet", "Gaillard", "Barbier", "Arnaud", "Martinez", "Gerard", "Roche", "Renard", "Schmitt", "Roy", "Leroux", "Colin", "Vidal", "Caron", "Picard", "Roger", "Fabre", "Aubert", "Lemoine", "Renaud", "Dumas", "Lacroix", "Olivier", "Philippe", "Bourgeois", "Pierre", "Benoit", "Rey", "Leclerc", "Payet", "Rolland", "Leclercq", "Guillaume", "Lecomte"]

#prénoms les plus donnés en France en 1900
prénom = ["Marie","Jean","Jeanne","Louis","Marguerite","Pierre","Joseph","Germaine","Henri","Louise","Marcel","Georges","André","Yvonne","Madeleine","Suzanne","Paul","René","Charles","François","Emile","Maurice","Marthe","Marcelle","Maria","Anne","Albert","Eugène","Joséphine","Léon","Eugénie","Berthe","Henriette","Lucien","Alice","Hélène","Lucie","Jules","Auguste","Georgette","Robert","Lucienne","Renée","Juliette","Fernand","Gaston","Raymond","Antoine","Roger","Gabrielle"]

In [3]:
#nombre de gens à créer
N = 100

In [4]:
def gen_nom():
    Prenom = prénom[int(random()*len(prénom))]
    Nom = Prenom
    while Prenom == Nom:
        Nom = nom[int(random()*len(nom))]
    return Prenom+' '+Nom

In [5]:
personne = []
for i in range(N):
    personne += [gen_nom()]

In [6]:
pd.DataFrame(personne,columns=["Nom"])

,Nom
0,Germaine Lacroix
1,Renée Andre
2,Marcelle Faure
3,Jeanne Roche
4,Joséphine Blanc
...,...
95,Joséphine Olivier
96,André Leclerc
97,René Perrin
98,Yvonne Bourgeois


In [7]:
pd.DataFrame(personne,columns=["Nom"]).to_excel("Humains.xlsx")

# Possibilité d'interroger la base

In [3]:
!pip install py2neo
# If you see warnings or errors when you run this command, re-run the command. It should run with no errors.

You should consider upgrading via the '/Users/maxime.capron/anaconda3/bin/python3 -m pip install --upgrade pip' command.


In [1]:
from py2neo import Graph

In [2]:
graph = Graph("bolt://3.90.65.37:34875", auth=("neo4j", "volumes-capes-retrievals"))

Qui reste-il à convaincre du projet D ?

In [6]:
exploratory_query = """
MATCH (p:Person)-[rel:EMPLOYE_DANS]-(c:Company)
WHERE p.position_D = "A convaincre"'
RETURN p.nom AS nom, c.titre AS entreprise, rel.position AS position
"""

graph.run(exploratory_query).to_data_frame()

,nom,entreprise,position
0,Anne Gautier,DCorp,Technique
1,Joseph Thomas,DCorp,Technique


Chercher les détracteurs d'un projet

In [10]:
def détracteurs(nom_du_projet):
    exploratory_query = """
    MATCH (p:Person)-[rel:EMPLOYE_DANS]-(c:Company)
    WHERE p.position_"""+nom_du_projet+""" = "A convaincre"
    RETURN p.nom AS nom, c.titre AS entreprise, rel.position AS position
    """

    return graph.run(exploratory_query).to_data_frame()

In [11]:
détracteurs("D")

,nom,entreprise,position
0,Anne Gautier,DCorp,Technique
1,Joseph Thomas,DCorp,Technique


Initialiser le graph

In [14]:
def initialize():
    
    code_initialisation = """

    // Supprimer tout ce qui préexiste pour avoir une feuille propre

    MATCH (n)
    DETACH DELETE n ;


    // Importer les noeuds 

    LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/MaximeCapron/foodforneo4j/master/fichiers%20csv/Humains.csv' AS row FIELDTERMINATOR ';'
    MERGE (p:Person {id: row.index})
    ON CREATE SET p.nom = row.Nom, p.position_A = row.Position_A, p.position_B = row.Position_B, p.position_C = row.Position_C, p.position_D = row.Position_D, p.position_E = row.Position_E;


    LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/MaximeCapron/foodforneo4j/master/fichiers%20csv/Entreprises.csv' AS row FIELDTERMINATOR ';'
    MERGE (c:Company {id: row.index})
    ON CREATE SET c.titre = row.Titre;


    // Importer les relations

    LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/MaximeCapron/foodforneo4j/master/fichiers%20csv/Relations1.csv' AS rel FIELDTERMINATOR ';'
    MATCH (p1 {id: rel.index})
    MATCH (p2 {id: rel.index_relation})
    MERGE (p1)-[r:CONNAISSANCE]->(p2)
    ON CREATE SET r.contexte = rel.contexte;


    LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/MaximeCapron/foodforneo4j/master/fichiers%20csv/Relations2.csv' AS rel FIELDTERMINATOR ';'
    MATCH (p1 {id: rel.index})
    MATCH (p2 {id: rel.index_relation})
    MERGE (p1)-[r:SUPERIEUR]->(p2) ;


    LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/MaximeCapron/foodforneo4j/master/fichiers%20csv/Relations3.csv' AS rel FIELDTERMINATOR ';'
    MATCH (c1 {id: rel.index})
    MATCH (c2 {id: rel.index_relation})
    MERGE (c1)-[r:FILIALE]->(c2) ;


    LOAD CSV WITH HEADERS FROM 'https://raw.githubusercontent.com/MaximeCapron/foodforneo4j/master/fichiers%20csv/Relations4.csv' AS rel FIELDTERMINATOR ';'
    MATCH (p {id: rel.index})
    MATCH (c {id: rel.index_relation})
    MERGE (p)-[r:EMPLOYE_DANS]->(c)
    ON CREATE SET r.position = rel.position

    """
    
    graph.run(code_initialisation)

Add or delete a node

In [21]:
def add_person(nom_person):
    
    code = """MERGE (p:Person {nom: """+nom_person+"""})"""
    graph.run(code)

In [ ]:
add_person("'Bob'")

In [27]:
graph.run("""
MATCH (p:Person {nom:'Bob'})
RETURN p.nom""").to_data_frame()

,p.nom
0,Bob
